### Importing packages

In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD,Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Fetching essential features

In [2]:
df=pd.read_csv(r"C:\Users\Micro\Downloads\archive (3)\Bank_Personal_Loan_Modelling.csv")
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
col=df.columns
col

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [4]:
col=['Age','Experience','Income','Family','CCAvg','Education','Mortgage','Securities Account','CD Account',\
 'Online','CreditCard','Personal Loan']
df=df[col]

### Checking for class imbalance

In [5]:
df["Personal Loan"].value_counts()

0    4520
1     480
Name: Personal Loan, dtype: int64

### Standardization

In [6]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

### Resolving class imbalance

In [8]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()
x,y=smote.fit_resample(x,y)

In [9]:
x.shape

(9040, 11)

In [10]:
y=y.reshape((9040,1))
y.shape

(9040, 1)

### Constructing NN

In [11]:
x=torch.tensor(x,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.float32)

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [13]:
model= nn.Sequential(nn.Linear(11,5) ,nn.Sigmoid(), nn.Linear(5,1), nn.Sigmoid())

In [14]:
loss_fn=nn.BCELoss()

In [15]:
for i in model.state_dict():
    print(i)

0.weight
0.bias
2.weight
2.bias


### Applying genetic algorithm
- Population : numbers that fall under standard normal curve
- Chromosomes/Individuals : weights to be optimised for the neural network
- Fitness function : the loss function whose value is to be minimised
- Cross over: single point crossover
- Mutation : adding noise to the weights assuming it to give better results

In [16]:
def generate_chromo(n):
    chromosomes=[]
    for i in range(n):
        t=np.random.normal(0,1,66)
        chromosomes.append(t)
    chromosomes=np.array(chromosomes)
    return chromosomes

In [17]:
def fitness(chromosomes,model,x,y):
    fit=[]
    for i in chromosomes:
        sd={}
        sd["0.weight"]=torch.tensor(i[:55].reshape(5,11))
        sd["0.bias"]=torch.tensor(i[55:60])
        sd["2.weight"]=torch.tensor(i[60:65].reshape(1,5))
        sd["2.bias"]=torch.tensor(i[65:])
        model.load_state_dict(sd)
        pred=model(x)
        loss=loss_fn(pred,y)
        fit.append(loss.item())
    fit=np.array(fit)
    ind=np.argsort(fit)
    print(np.amin(fit))
    return chromosomes[ind[:4]]

In [18]:
def crossover(fit_chromo):
    new=[]
    for i in range(0,fit_chromo.shape[0],2):
        t1=fit_chromo[i][:33]
        t2=fit_chromo[i][33:]
        t3=fit_chromo[i+1][:33]
        t4=fit_chromo[i+1][33:]
        temp1=np.append(t1,t4)
        temp2=np.append(t3,t2)
        new.append(temp1)
        new.append(temp2)
    return np.array(new)

In [19]:
def mutation(fit_chromo):
    for i in range(fit_chromo.shape[0]):
        idx=np.random.randint(low=0,high=66,size=(1,30))
        noise=np.random.randint(low=-1,high=1,size=(1,30))
        noise=noise.astype("float")/100
        for j in range(len(idx)):
            fit_chromo[i,idx[j]]+=noise[j]
        return fit_chromo

In [28]:
generations=80
for i in range(generations):
    if(i==0):
        chromo=generate_chromo(10)
    else:
        chromo=np.append(fit_chromo,generate_chromo(2)) #4 offsprings and 2 random
        chromo=chromo.reshape(6,66)
    fit_chromo=fitness(chromo,model,x,y)
    fit_chromo=crossover(fit_chromo)
    fit_chromo=mutation(fit_chromo)
    
#Loss values over generations:

0.7429754734039307
0.5486286282539368
0.6238099932670593
0.6229928731918335
0.5488134026527405
0.6234692335128784
0.5491083860397339
0.6238861680030823
0.5496754050254822
0.6645413041114807
0.7110646963119507
0.6478090286254883
0.702497661113739
0.668057382106781
0.6352988481521606
0.6746240854263306
0.6352784633636475
0.6749588251113892
0.6356906890869141
0.6750803589820862
0.635975182056427
0.6273770928382874
0.6360623836517334
0.666666567325592
0.6820358037948608
0.6791265606880188
0.6360504627227783
0.6422464847564697
0.603223979473114
0.6494840979576111
0.6177946925163269
0.6496745347976685
0.5253025889396667
0.6030543446540833
0.6492468118667603
0.5250791311264038
0.602475643157959
0.647662341594696
0.5250300168991089
0.6022347211837769
0.5249078273773193
0.602240264415741
0.6314452290534973
0.6372807621955872
0.5364819169044495
0.5255750417709351
0.6239591240882874
0.5360823273658752
0.5258689522743225
0.5683894157409668
0.5259986519813538
0.5684113502502441
0.526064395904541
0.